In [11]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURAÇÃO ---
WEIGHTS = {
    'Shot_Stopping': 0.70,   
    'Distribution': 0.15,    
    'Aeriel_Control': 0.15   
}

NAME_MAPPING = {
    'Brighton': 'Brighton & Hove Albion', 'Brighton & Hove Albion': 'Brighton',
    'Manchester Utd': 'Manchester United', 'Manchester United': 'Manchester Utd',
    'Newcastle Utd': 'Newcastle United', 'Newcastle United': 'Newcastle Utd',
    'Nott\'ham Forest': 'Nottingham Forest', 'Nottingham Forest': 'Nott\'ham Forest',
    'Tottenham': 'Tottenham Hotspur', 'Tottenham Hotspur': 'Tottenham',
    'West Ham': 'West Ham United', 'West Ham United': 'West Ham',
    'Wolves': 'Wolverhampton Wanderers', 'Wolverhampton Wanderers': 'Wolves'
}

def load_and_clean_data():
    # Ajuste o caminho conforme necessário
    path = "data/processed/pl_goalkeepers_2526_matches.csv"
    if not os.path.exists(path): 
        # Tenta procurar na raiz se não achar na pasta processed
        if os.path.exists("pl_goalkeepers_2526_matches.csv"):
            path = "pl_goalkeepers_2526_matches.csv"
        else:
            raise FileNotFoundError(f"Arquivo {path} não existe.")
    
    df = pd.read_csv(path)
    # Limpa sufixos indesejados nas colunas se houver
    df.columns = [c.rstrip('_') for c in df.columns]
    
    rename_map = {
        'team': 'Team', 'player': 'Player', 'game': 'Game', 
        'date': 'Date', 'min': 'Minutes', 'psxg': 'PSxG', 'psxg_net': 'PSxG_Net',
        'opa': 'OPA'
    }
    df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})
    
    if 'Date' not in df.columns and 'Game' in df.columns:
        # Tenta extrair data da string do jogo se necessário
        try:
            df['Date'] = pd.to_datetime(df['Game'].str.split(' ').str[0])
        except:
            pass
        
    return df

def get_opponent(row):
    try:
        if 'Game' not in row: return "Unknown"
        match_str = row['Game'].split(' ', 1)[1]
        teams = match_str.split('-')
        my_team = row['Team']
        for t in teams:
            t = t.strip()
            # Verifica se o time analisado é o "meu time"
            is_me = (t == my_team) or (NAME_MAPPING.get(t) == my_team)
            if not is_me: return NAME_MAPPING.get(t, t)
        return "Unknown"
    except: return "Unknown"

def normalize_series(series):
    return series.rank(pct=True) * 100

def aggregate_and_rank(df):
    print("⚙️ Calculando Ranking Híbrido (Volume + Eficiência)...")
    
    df['Opponent'] = df.apply(get_opponent, axis=1)
    
    # Colunas necessárias para agregação (garante que existem)
    agg_dict = {
        'Team': 'last',
        'PSxG_Net': 'sum',
        'PSxG': 'sum',      # Perigo Total Enfrentado
        'OPA': 'sum',
        'Minutes': 'sum',
        'Game': 'count'
    }
    
    # Adiciona colunas opcionais se existirem
    if 'Crosses_Stopped' in df.columns: agg_dict['Crosses_Stopped'] = 'sum'
    if 'Crosses_Faced' in df.columns: agg_dict['Crosses_Faced'] = 'sum'
    if 'Launch_Completed' in df.columns: agg_dict['Launch_Completed'] = 'sum'
    if 'Launch_Attempted' in df.columns: agg_dict['Launch_Attempted'] = 'sum'

    # Agregação por Jogador
    grouped = df.groupby(['Player']).agg(agg_dict).reset_index()
    
    grouped.rename(columns={'Game': 'Games_Played'}, inplace=True)
    
    # Filtro Mínimo de Jogos
    ranked = grouped[grouped['Games_Played'] >= 5].copy()
    
    # --- MÉTRICAS DE SHOT STOPPING (A MUDANÇA) ---
    
    # 1. Volume: Gols evitados por 90 min (Favorece quem trabalha muito - Times Pequenos)
    ranked['PSxG_Net_p90'] = (ranked['PSxG_Net'] / ranked['Minutes']) * 90
    
    # 2. Eficiência: % do Perigo Neutralizado (Favorece quem trabalha pouco mas bem - Times Grandes)
    # Fórmula: PSxG Net / PSxG Total
    ranked['Prevention_Ratio'] = np.where(ranked['PSxG'] > 0, 
                                          ranked['PSxG_Net'] / ranked['PSxG'], 
                                          0)

    # --- MÉTRICAS GERAIS ---
    if 'Crosses_Faced' in ranked.columns:
        ranked['Cross_Stop_Pct'] = np.where(ranked['Crosses_Faced'] > 0, 
                                            ranked['Crosses_Stopped'] / ranked['Crosses_Faced'], 0)
    else:
        ranked['Cross_Stop_Pct'] = 0

    if 'Launch_Attempted' in ranked.columns:
         ranked['Launch_Completion_pct'] = np.where(ranked['Launch_Attempted'] > 0, 
                                            ranked['Launch_Completed'] / ranked['Launch_Attempted'], 0)
    
    ranked['OPA_p90'] = (ranked['OPA'] / ranked['Minutes']) * 90
    
    # --- NORMALIZAÇÃO (0-100) ---
    score_volume = normalize_series(ranked['PSxG_Net_p90'])
    score_efficiency = normalize_series(ranked['Prevention_Ratio'])
    
    # COMBINAÇÃO DO SHOT STOPPING: 50% Volume / 50% Eficiência
    ranked['Score_Shot_Stopping'] = (score_volume * 0.5) + (score_efficiency * 0.5)
    
    ranked['Score_Aerial'] = normalize_series(ranked['Cross_Stop_Pct'])
    
    # Distribuição: Média entre OPA (Saídas) e Lançamentos (se houver)
    if 'Launch_Completion_pct' in ranked.columns:
         ranked['Score_Distribution'] = (normalize_series(ranked['OPA_p90']) + normalize_series(ranked['Launch_Completion_pct'])) / 2
    else:
         ranked['Score_Distribution'] = normalize_series(ranked['OPA_p90'])
    
    # Score Final
    ranked['Final_Score'] = (
        (ranked['Score_Shot_Stopping'] * WEIGHTS['Shot_Stopping']) +
        (ranked['Score_Aerial'] * WEIGHTS['Aeriel_Control']) +
        (ranked['Score_Distribution'] * WEIGHTS['Distribution'])
    )
    
    # Bônus de Consistência (Premia quem jogou mais)
    ranked['Final_Score'] += (ranked['Games_Played'] / ranked['Games_Played'].max()) * 5
    
    ranked = ranked.sort_values('Final_Score', ascending=False)
    ranked['Rank'] = range(1, len(ranked) + 1)
    
    print("\n🏆 TOP 10 RANKING (Ajustado para Times Grandes):")
    # Mostra as duas colunas para compararmos
    cols = ['Rank', 'Player', 'Team', 'Final_Score', 'PSxG_Net_p90', 'Prevention_Ratio']
    print(ranked[cols].head(10).to_string(index=False))
    
    return ranked

if __name__ == "__main__":
    df_matches = load_and_clean_data()
    df_ranking = aggregate_and_rank(df_matches)
    
    # Garante que o diretório existe
    os.makedirs("data/processed", exist_ok=True)
    
    # Salva o arquivo completo para o App usar
    df_ranking.to_csv("data/processed/final_ranking_table.csv", index=False)
    print("\nArquivo salvo com sucesso em: data/processed/final_ranking_table.csv")

⚙️ Calculando Ranking Híbrido (Volume + Eficiência)...

🏆 TOP 10 RANKING (Ajustado para Times Grandes):
 Rank               Player                   Team  Final_Score  PSxG_Net_p90  Prevention_Ratio
    1          Robin Roefs             Sunderland    92.962963      0.147619          0.123506
    2       Robert Sánchez                Chelsea    83.465608      0.052509          0.045455
    3       Dean Henderson         Crystal Palace    83.333333      0.180952          0.141791
    4    Emiliano Martínez            Aston Villa    83.121693      0.121212          0.111111
    5      Jordan Pickford                Everton    82.222222      0.123810          0.094203
    6    Guglielmo Vicario      Tottenham Hotspur    75.370370      0.019048          0.014599
    7            Nick Pope       Newcastle United    74.497354     -0.080838         -0.075949
    8      Bart Verbruggen Brighton & Hove Albion    73.333333      0.042857          0.031142
    9 Gianluigi Donnarumma        Manches